In [1]:
import numpy as np
import pandas as pd
from scipy.stats import pearsonr, spearmanr
import statsmodels.api

In [4]:
plac_tpm_mtx

,Unnamed: 0,X16680_brain_S65_tpm,X16681_brain_S67_tpm,X16682_brain_S63_tpm,X16730_brain_S31_tpm,X16737_brain_S25_tpm,X16849_brain_S55_tpm,X17255_brain_S23_tpm,X17257_brain_S19_tpm,X17259_brain_S21_tpm,...,X20336_brain_S3_tpm,X20381_brain_S51_tpm,X20385_brain_S73_tpm,X20442_brain_S49_tpm,X20748_brain_S45_tpm,X20815_brain_S9_tpm,X20818_brain_S7_tpm,X20819_brain_S11_tpm,X20820_brain_S13_tpm,X21027_brain_S2_tpm
0,TSPAN6,7.540561,13.717186,9.651405,7.701185,10.384579,4.367738,5.034751,8.122025,13.464602,...,11.641640,10.919947,10.502449,10.751905,10.898931,13.645226,6.783930,5.809494,13.000301,12.176359
1,DPM1,11.074675,11.021560,11.138585,11.666419,9.886661,10.269420,8.915343,10.683705,10.798776,...,12.177185,16.196006,10.915745,9.459815,11.419796,10.882772,8.700432,8.184809,8.145981,9.341398
2,SCYL3,15.586435,9.308029,9.475194,10.635434,9.604278,15.864695,11.426068,10.366156,11.630050,...,11.902583,21.589225,10.828441,11.013989,12.361399,10.244234,9.885108,9.653065,10.966263,9.418808
3,C1orf112,12.201671,9.612893,9.279576,10.829776,9.582145,17.692784,8.851946,12.463479,11.285813,...,21.456186,30.526567,10.696870,12.349168,18.534506,9.513623,7.915236,9.029957,11.049305,10.161578
4,FGR,5.177579,2.873644,3.356018,2.576223,2.513543,1.123936,3.733574,2.806599,2.731075,...,4.208339,14.162436,6.841087,6.033511,1.490016,3.361757,5.089527,5.487517,5.098597,4.149720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18645,C8orf44,5.126182,3.381196,4.351159,3.965751,2.634980,5.931754,4.107236,4.656206,4.744800,...,3.252409,4.943401,5.146342,3.934214,4.997414,3.948472,4.732170,3.495878,3.701566,3.903702
18646,ACTL10,4.572143,2.229389,3.300459,3.600729,2.842972,3.201643,4.973062,4.065191,3.754177,...,3.367618,3.247128,2.373019,2.793562,2.479658,3.091900,5.077728,5.227497,4.418817,2.917091
18647,PRRC2B,358.965074,302.997392,384.112831,371.198346,419.872326,586.246727,569.903341,483.429400,377.440600,...,415.724391,176.611223,325.123313,374.670119,403.252349,378.030840,414.323417,440.909381,372.832455,406.767792
18648,F8A2,20.994905,31.870412,33.562239,34.536503,32.516254,20.176175,34.748379,35.531188,32.046262,...,21.595753,16.561877,29.108438,23.315685,22.715635,36.558047,29.109389,35.628689,24.777544,34.583962


In [29]:
plac_tpm_mtx = pd.read_csv('C:\\Users\\Ben\\OneDrive - Duke University\\bilbo_lab\\alexis_sequencing\\counts\\placenta_tpm_TRIM.csv')
plac_tpm_mtx.set_index('Unnamed: 0', inplace = True)
genes = plac_tpm_mtx.index.values
#plac_tpm_mtx.drop(columns = {'Length', 'Geneid', 'X', 'Unnamed: 0'}, inplace = True)
plac_tpm_mtx_t = plac_tpm_mtx.T


sampleID = []
region = []

for i in plac_tpm_mtx.columns:
    if 'X' in i:
        sampleID = np.append(sampleID, i.split('_')[0].split('X')[1])
        region = np.append(region, i.split('_')[1])
        
plac_tpm_mtx_t['sampleID'] = sampleID
plac_tpm_mtx_t['region'] = region
plac_tpm_mtx_t.set_index('sampleID', inplace = True)

In [30]:
metadata = pd.read_csv('C:\\Users\\Ben\\OneDrive - Duke University\\bilbo_lab\\alexis_sequencing\\metadata\\human_seq_metadata.csv')

In [31]:
sampleID = []

for i in metadata['Sample ID']:
    sampleID = np.append(sampleID, i.split('-')[1])
    
metadata['sampleID'] = sampleID
metadata.set_index('sampleID', inplace = True)

In [32]:
#### just gonna get a broad overall correlation

final_df = metadata.join(plac_tpm_mtx_t)
final_df = final_df[final_df['exclude'] == 'n']

In [33]:
pearson_df = pd.DataFrame(columns = ['Gene', 'CorrCoef', 'pval'])

for gene in genes:
    try:
        corr = spearmanr(final_df[gene], final_df['trig'])
        coef = corr[0]
        pval = corr[1]
        
        int_df = pd.DataFrame(data = {'Gene': [gene], 'CorrCoef': [coef], 'pval': [pval]})

        pearson_df = pearson_df.append(int_df)
        
    except:
        pass
    
    
# pearson_df.sort_values(by = 'pval').to_csv('pearson_corr_w_triglycerides.csv')
# pearson_df.sort_values(by = 'pval', inplace = True)

pearson_df_negs = pearson_df[pearson_df['CorrCoef'] < 0]
pearson_df_pos = pearson_df[pearson_df['CorrCoef'] > 0]

# pearson_df_pos.reset_index()[0:100].to_csv('pearson_corr_w_triglycerides_pos_t100.csv')
# pearson_df_negs.reset_index()[0:100].to_csv('pearson_corr_w_triglycerides_negs_t100.csv')

# pearson_df_pos[pearson_df_pos['pval'] < .05][0:int(len(pearson_df_pos[pearson_df_pos['pval'] < .05])*.10)].to_csv('pearson_corr_w_triglycerides_pos_10p.csv')
# pearson_df_negs[pearson_df_negs['pval'] < .05][0:int(len(pearson_df_negs[pearson_df_negs['pval'] < .05])*.10)].to_csv('pearson_corr_w_triglycerides_negs_10p.csv')

## saving the sig genes
pearson_df_pos[pearson_df_pos['pval'] < .05].to_csv('pearson_corr_w_triglycerides_placenta_pos_sig_TPMTRIM.csv')
pearson_df_negs[pearson_df_negs['pval'] < .05].to_csv('pearson_corr_w_triglycerides_placenta_negs_sig_TPMTRIM.csv')

## saving the CORRELATED genes
# pearson_df_pos[pearson_df_pos['CorrCoef'] > .5].to_csv('pearson_corr_w_triglycerides_placenta_pos_corr.csv')

# pearson_df_negs['abs'] = np.abs(pearson_df_negs['CorrCoef'])
# pearson_df_negs[pearson_df_negs['abs'] > .5].to_csv('pearson_corr_w_triglycerides_placenta_negs_corr.csv')


print('# of significant + correlations', len(pearson_df_pos[pearson_df_pos['pval'] < .05]))
print('# of significant - correlations', len(pearson_df_negs[pearson_df_negs['pval'] < .05]))

## printing the number of genes that are correlated about .5
# print('# of strong + correlations', len(pearson_df_pos[pearson_df_pos['CorrCoef'] > .5]))
# print('# of strong - correlations', len(pearson_df_negs[pearson_df_negs['abs'] > .5]))

# of significant + correlations 611
# of significant - correlations 55


In [34]:
#### just correlating in the female placenta

final_df = metadata.join(plac_tpm_mtx_t)
final_df = final_df[final_df['exclude'] == 'n']
final_df = final_df[final_df['sex'] == 'Female']
final_df = final_df[final_df['region'] == 'placenta']

In [35]:
pearson_df = pd.DataFrame(columns = ['Gene', 'CorrCoef', 'pval'])

for gene in genes:
    try:
        corr = spearmanr(final_df[gene], final_df['trig'])
        coef = corr[0]
        pval = corr[1]
        
        int_df = pd.DataFrame(data = {'Gene': [gene], 'CorrCoef': [coef], 'pval': [pval]})

        pearson_df = pearson_df.append(int_df)
        
    except:
        pass
    
    
pearson_df.sort_values(by = 'pval').to_csv('pearson_corr_w_triglycerides_f_plac.csv')
pearson_df.sort_values(by = 'pval', inplace = True)

pearson_df_negs = pearson_df[pearson_df['CorrCoef'] < 0]
pearson_df_pos = pearson_df[pearson_df['CorrCoef'] > 0]


# pearson_df_pos.reset_index()[0:100].to_csv('pearson_corr_w_triglycerides_f_plac_pos_t100.csv')
# pearson_df_negs.reset_index()[0:100].to_csv('pearson_corr_w_triglycerides_f_plac_negs_t100.csv')

# ## saving the top 10% of genes
# pearson_df_pos[pearson_df_pos['pval'] < .05][0:int(len(pearson_df_pos[pearson_df_pos['pval'] < .05])*.10)].to_csv('pearson_corr_w_triglycerides_f_plac_pos_10p.csv')
# pearson_df_negs[pearson_df_negs['pval'] < .05][0:int(len(pearson_df_negs[pearson_df_negs['pval'] < .05])*.10)].to_csv('pearson_corr_w_triglycerides_f_plac_negs_10p.csv')

## saving the sig genes
pearson_df_pos[pearson_df_pos['pval'] < .05].to_csv('pearson_corr_w_triglycerides_f_plac_pos_sig_TPMTRIM.csv')
pearson_df_negs[pearson_df_negs['pval'] < .05].to_csv('pearson_corr_w_triglycerides_f_plac_negs_sig_TPMTRIM.csv')

## saving the CORRELATED genes
# pearson_df_pos[pearson_df_pos['CorrCoef'] > .5].to_csv('pearson_corr_w_triglycerides_f_plac_pos_corr.csv')

# pearson_df_negs['abs'] = np.abs(pearson_df_negs['CorrCoef'])
# pearson_df_negs[pearson_df_negs['abs'] > .5].to_csv('pearson_corr_w_triglycerides_f_plac_negs_corr.csv')


print('# of significant + correlations', len(pearson_df_pos[pearson_df_pos['pval'] < .05]))
print('# of significant - correlations', len(pearson_df_negs[pearson_df_negs['pval'] < .05]))

# ## printing the number of genes that are correlated about .5
# print('# of strong + correlations', len(pearson_df_pos[pearson_df_pos['CorrCoef'] > .5]))
# print('# of strong - correlations', len(pearson_df_negs[pearson_df_negs['abs'] > .5]))

# of significant + correlations 93
# of significant - correlations 1080


In [36]:
#### just correlating in the male placenta

final_df = metadata.join(plac_tpm_mtx_t)
final_df = final_df[final_df['exclude'] == 'n']
final_df = final_df[final_df['sex'] == 'Male']
final_df = final_df[final_df['region'] == 'placenta']

In [39]:
%%time

pearson_df = pd.DataFrame(columns = ['Gene', 'CorrCoef', 'pval'])

for gene in genes:
    try:
        corr = pearsonr(final_df[gene], final_df['trig'])
        coef = corr[0]
        pval = corr[1]
        
        int_df = pd.DataFrame(data = {'Gene': [gene], 'CorrCoef': [coef], 'pval': [pval]})

        pearson_df = pearson_df.append(int_df)
        
    except:
        pass
    
    
pearson_df.sort_values(by = 'pval').to_csv('pearson_corr_w_triglycerides_m_plac_BEN.csv')
pearson_df.sort_values(by = 'pval', inplace = True)

pearson_df_negs = pearson_df[pearson_df['CorrCoef'] < 0]
pearson_df_pos = pearson_df[pearson_df['CorrCoef'] > 0]

# pearson_df_pos.reset_index()[0:100].to_csv('pearson_corr_w_triglycerides_m_plac_pos_t100.csv')
# pearson_df_negs.reset_index()[0:100].to_csv('pearson_corr_w_triglycerides_m_plac_negs_t100.csv')

# ## saving the top 10% of genes
# pearson_df_pos[pearson_df_pos['pval'] < .05][0:int(len(pearson_df_pos[pearson_df_pos['pval'] < .05])*.10)].to_csv('pearson_corr_w_triglycerides_m_plac_pos_10p.csv')
# pearson_df_negs[pearson_df_negs['pval'] < .05][0:int(len(pearson_df_negs[pearson_df_negs['pval'] < .05])*.10)].to_csv('pearson_corr_w_triglycerides_m_plac_negs_10p.csv')


## saving the sig genes
pearson_df_pos[pearson_df_pos['pval'] < .05].to_csv('pearson_corr_w_triglycerides_m_plac_pos_sig_TPMTRIM.csv')
pearson_df_negs[pearson_df_negs['pval'] < .05].to_csv('pearson_corr_w_triglycerides_m_plac_negs_sig_TPMTRIM.csv')

# ## saving the CORRELATED genes
# pearson_df_pos[pearson_df_pos['CorrCoef'] > .5].to_csv('pearson_corr_w_triglycerides_m_plac_pos_corr.csv')

# pearson_df_negs['abs'] = np.abs(pearson_df_negs['CorrCoef'])
# pearson_df_negs[pearson_df_negs['abs'] > .5].to_csv('pearson_corr_w_triglycerides_m_plac_negs_corr.csv')


print('# of significant + correlations', len(pearson_df_pos[pearson_df_pos['pval'] < .05]))
print('# of significant - correlations', len(pearson_df_negs[pearson_df_negs['pval'] < .05]))

## printing the number of genes that are correlated about .5
# print('# of strong + correlations', len(pearson_df_pos[pearson_df_pos['CorrCoef'] > .5]))
# print('# of strong - correlations', len(pearson_df_negs[pearson_df_negs['abs'] > .5]))

# of significant + correlations 2821
# of significant - correlations 173
Wall time: 16.8 s


In [40]:
br_tpm_mtx = pd.read_csv('C:\\Users\\Ben\\OneDrive - Duke University\\bilbo_lab\\alexis_sequencing\\counts\\brain_tpm_TRIM.csv')
br_tpm_mtx.set_index('Unnamed: 0', inplace = True)
genes = br_tpm_mtx.index.values
#br_tpm_mtx.drop(columns = {'Length', 'Geneid', 'X', 'Unnamed: 0'}, inplace = True)
br_tpm_mtx_t = br_tpm_mtx.T


sampleID = []
region = []

for i in br_tpm_mtx.columns:
    if 'X' in i:
        sampleID = np.append(sampleID, i.split('_')[0].split('X')[1])
        region = np.append(region, i.split('_')[1])
        
br_tpm_mtx_t['sampleID'] = sampleID
br_tpm_mtx_t['region'] = region
br_tpm_mtx_t.set_index('sampleID', inplace = True)

In [41]:
#### just correlating in the female brain

final_df = metadata.join(br_tpm_mtx_t)
final_df = final_df[final_df['exclude'] == 'n']
final_df = final_df[final_df['sex'] == 'Female']
final_df = final_df[final_df['region'] == 'brain']

In [42]:
pearson_df = pd.DataFrame(columns = ['Gene', 'CorrCoef', 'pval'])

for gene in genes:
    try:
        corr = pearsonr(final_df[gene], final_df['trig'])
        coef = corr[0]
        pval = corr[1]
        
        int_df = pd.DataFrame(data = {'Gene': [gene], 'CorrCoef': [coef], 'pval': [pval]})

        pearson_df = pearson_df.append(int_df)
        
    except:
        pass
    
    
pearson_df.sort_values(by = 'pval').to_csv('pearson_corr_w_triglycerides_f_brain_BEN.csv')

pearson_df.sort_values(by = 'pval', inplace = True)

pearson_df_negs = pearson_df[pearson_df['CorrCoef'] < 0]
pearson_df_pos = pearson_df[pearson_df['CorrCoef'] > 0]

## saving the top 100 genes
# pearson_df_pos.reset_index()[0:100].to_csv('pearson_corr_w_triglycerides_f_brain_pos_t100.csv')
# pearson_df_negs.reset_index()[0:100].to_csv('pearson_corr_w_triglycerides_f_brain_negs_t100.csv')

# ## saving the top 10% of genes
# pearson_df_pos[pearson_df_pos['pval'] < .05][0:int(len(pearson_df_pos[pearson_df_pos['pval'] < .05])*.10)].to_csv('pearson_corr_w_triglycerides_f_brain_pos_10p.csv')
# pearson_df_negs[pearson_df_negs['pval'] < .05][0:int(len(pearson_df_negs[pearson_df_negs['pval'] < .05])*.10)].to_csv('pearson_corr_w_triglycerides_f_brain_negs_10p.csv')

## saving the sig genes
pearson_df_pos[pearson_df_pos['pval'] < .05].to_csv('pearson_corr_w_triglycerides_f_brain_pos_sig_TPMTRIM.csv')
pearson_df_negs[pearson_df_negs['pval'] < .05].to_csv('pearson_corr_w_triglycerides_f_brain_negs_sig_TPMTRIM.csv')


## saving the CORRELATED genes
# pearson_df_pos[pearson_df_pos['CorrCoef'] > .5].to_csv('pearson_corr_w_triglycerides_f_brain_pos_corr.csv')

# pearson_df_negs['abs'] = np.abs(pearson_df_negs['CorrCoef'])
# pearson_df_negs[pearson_df_negs['abs'] > .5].to_csv('pearson_corr_w_triglycerides_f_brain_negs_corr.csv')

## printing the number of genes that came up significant WITHOUT any correction
print('# of significant + correlations', len(pearson_df_pos[pearson_df_pos['pval'] < .05]))
print('# of significant - correlations', len(pearson_df_negs[pearson_df_negs['pval'] < .05]))


## printing the number of genes that are correlated about .5
# print('# of strong + correlations', len(pearson_df_pos[pearson_df_pos['CorrCoef'] > .5]))
# print('# of strong - correlations', len(pearson_df_negs[pearson_df_negs['abs'] > .5]))

# of significant + correlations 596
# of significant - correlations 185


In [43]:
#### just correlating in the male brain

final_df = metadata.join(br_tpm_mtx_t)
final_df = final_df[final_df['exclude'] == 'n']
final_df = final_df[final_df['sex'] == 'Male']
final_df = final_df[final_df['region'] == 'brain']

In [44]:
pearson_df = pd.DataFrame(columns = ['Gene', 'CorrCoef', 'pval'])

for gene in genes:
    try:
        corr = pearsonr(final_df[gene], final_df['trig'])
        coef = corr[0]
        pval = corr[1]
        
        int_df = pd.DataFrame(data = {'Gene': [gene], 'CorrCoef': [coef], 'pval': [pval]})

        pearson_df = pearson_df.append(int_df)
        
    except:
        pass
    
    
pearson_df.sort_values(by = 'pval').to_csv('pearson_corr_w_triglycerides_m_brain_BEN.csv')
pearson_df.sort_values(by = 'pval', inplace = True)

pearson_df_negs = pearson_df[pearson_df['CorrCoef'] < 0]
pearson_df_pos = pearson_df[pearson_df['CorrCoef'] > 0]


# pearson_df_pos.reset_index()[0:100].to_csv('pearson_corr_w_triglycerides_m_brain_pos_t100.csv')
# pearson_df_negs.reset_index()[0:100].to_csv('pearson_corr_w_triglycerides_m_brain_negs_t100.csv')

# ## saving the top 10% of genes
# pearson_df_pos[pearson_df_pos['pval'] < .05][0:int(len(pearson_df_pos[pearson_df_pos['pval'] < .05])*.10)].to_csv('pearson_corr_w_triglycerides_m_brain_pos_10p.csv')
# pearson_df_negs[pearson_df_negs['pval'] < .05][0:int(len(pearson_df_negs[pearson_df_negs['pval'] < .05])*.10)].to_csv('pearson_corr_w_triglycerides_m_brain_negs_10p.csv')

## saving the sig genes
pearson_df_pos[pearson_df_pos['pval'] < .05].to_csv('pearson_corr_w_triglycerides_m_brain_pos_sig_TPMTRIM.csv')
pearson_df_negs[pearson_df_negs['pval'] < .05].to_csv('pearson_corr_w_triglycerides_m_brain_negs_sig_TPMTRIM.csv')


## saving the CORRELATED genes
# pearson_df_pos[pearson_df_pos['CorrCoef'] > .5].to_csv('pearson_corr_w_triglycerides_m_brain_pos_corr.csv')

# pearson_df_negs['abs'] = np.abs(pearson_df_negs['CorrCoef'])
# pearson_df_negs[pearson_df_negs['abs'] > .5].to_csv('pearson_corr_w_triglycerides_m_brain_negs_corr.csv')

print('# of significant + correlations', len(pearson_df_pos[pearson_df_pos['pval'] < .05]))
print('# of significant - correlations', len(pearson_df_negs[pearson_df_negs['pval'] < .05]))

## printing the number of genes that are correlated about .5
# print('# of strong + correlations', len(pearson_df_pos[pearson_df_pos['CorrCoef'] > .5]))
# print('# of strong - correlations', len(pearson_df_negs[pearson_df_negs['abs'] > .5]))

# of significant + correlations 23
# of significant - correlations 22
